# **REGRESSÃO LINEAR**
*   ### **Case**: *Livraria*



### Instalação de bibliotecas

In [ ]:
# As bibliotecas utilizadas neste notebook já vêm pré-instaladas no Google Colab

### Carregamento de bibliotecas

In [ ]:
from IPython.display import display
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf                                           # Ajuste de modelos de regressão
import statsmodels.api as sm                                                    # Gráfico de Q-Q plot
from statsmodels.stats.outliers_influence import variance_inflation_factor      # Cálculo do VIF

### Leitura da base de dados

In [ ]:
dados_estoque = pd.read_table("Livraria_Estoque.txt", sep = "\t")

### Visualização da base de dados

In [ ]:
display(dados_estoque)

### Análise bivariada: variável resposta vs. variáveis explicativas

*Gráficos de dispersão para variáveis explicativas quantitativas*


In [ ]:
sns.pairplot(dados_estoque[['qtde_paginas', 'preco']],
             plot_kws = {'color':'darkturquoise'},
             diag_kws = {'color':'darkturquoise'})

plt.suptitle("Correlograma", y = 1.05)

plt.show()

*Matriz de correlações*

In [ ]:
dados_estoque[['qtde_paginas', 'preco']].corr()

*Boxpots para variáveis explicativas qualitativas*

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 3, figsize = (12, 6))

variaveis = ['categoria', 'edicao', 'tipo_capa', 'densidade_papel', 'cor', 'ano']

for ax, variavel in zip(axes.flat, variaveis):

  sns.boxplot(y = variavel,
              x = 'preco',
              data = dados_estoque,
              width = 0.4,
              boxprops = {'facecolor':'darkturquoise', 'edgecolor':'white'},
              whiskerprops = {'color':'turquoise'},
              capprops     = {'color': 'darkturquoise'},
              medianprops  = {'color':'white'},
              flierprops   = {'markerfacecolor': 'white', 'markeredgecolor': 'darkturquoise', 'marker': 'o', 'markersize': 3, 'color': 'turquoise', 'linestyle': 'none'},
              ax = ax)
  ax.set_title(f'{variavel} vs. preço')

plt.tight_layout()
plt.show()

### Modelo de regressão linear múltipla

*Ajuste do modelo*

In [ ]:
modelo = smf.ols('preco ~ categoria + qtde_paginas + edicao + tipo_capa	+ densidade_papel	+ cor +	ano',
                 data = dados_estoque).fit()

print(modelo.summary())

*Avaliação de colinearidade*

In [ ]:
vif = pd.DataFrame({
    "Variável": modelo.model.exog_names,
    "VIF": [variance_inflation_factor(modelo.model.exog, i) for i in range(modelo.model.exog.shape[1])]
})

print(vif)

### Qualidade de ajuste

In [ ]:
# Coeficiente de determinação ajustado (R² ajustado)
modelo.rsquared

In [ ]:
# MAE
np.mean(np.abs(dados_estoque['preco'] - modelo.predict(dados_estoque)))

In [ ]:
# MAPE
np.mean(np.abs((dados_estoque['preco'] - modelo.predict(dados_estoque)) / dados_estoque['preco']))

### Análise de resíduos

In [ ]:
# Histograma dos resíduos
plt.hist(modelo.resid,
         bins = 30,
         color = "darkturquoise",
         edgecolor = "white")
plt.title("Histograma dos resíduos")
plt.show()

In [ ]:
# QQ plot dos resíduos
sm.qqplot(modelo.resid,
          line = '45',
          fit = True,
          markerfacecolor = 'darkturquoise',
          markeredgecolor = 'darkturquoise')
plt.title("QQ-plot dos resíduos")
plt.show()

In [ ]:
# Resíduos vs. valores preditos
plt.scatter(modelo.predict(dados_estoque),
            modelo.resid,
            color = "darkturquoise",
            alpha = 0.4)
plt.xlabel("Valores preditos")
plt.ylabel("Resíduos")
plt.title("Resíduos vs. valores preditos")
plt.show()

### Exemplo de uso (aplicação) do modelo

In [ ]:
# Criação de base de dados com um novo título
# Obs.: os nomes das colunas devem ser idênticos aos da base utilizada para construção do modelo
novos_dados = pd.DataFrame({
    'categoria'      : ['Autoajuda'],
    'qtde_paginas'   : [500],
    'edicao'         : ['Original'],
    'tipo_capa'      : ['Flexivel'],
    'densidade_papel': ['Baixa'],
    'cor'            : ['Preto'],
    'ano'            : ['2021-2023']
})

In [ ]:
# Aplicação do modelo
modelo.predict(novos_dados)